not good


random forest for refined dataset that selects features with small p-value
- imputed with mean value of each column
- eliminate outliers and leverges after normalized
- select features with small p-value

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

import scipy

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)

## load data

In [4]:
pth_data = './processed_data/train_selected_not_norm.csv'
df_selected = pd.read_csv(pth_data)

In [5]:
df_selected

,height,weight,gender,bmi,rh,stress_level,skin_temp,TC,TS,Clo,env_temp
0,1.60,52.5,1.0,20.507812,62.985,19.500000,32.003449,4.0,4.0,0.61,75.942
1,1.77,58.0,-1.0,18.513199,48.702,2.000000,30.933000,4.0,4.0,0.36,77.378
2,1.71,64.0,-1.0,21.887076,47.990,33.000000,33.199000,4.0,4.0,0.61,75.484
3,1.77,58.0,-1.0,18.513199,52.445,53.000000,30.431000,4.0,4.0,0.36,77.203
4,1.77,58.0,-1.0,18.513199,56.396,80.000000,30.087000,4.0,4.0,0.36,77.902
...,...,...,...,...,...,...,...,...,...,...,...
2282,1.77,90.0,-1.0,28.727377,59.397,33.000000,31.353000,4.0,4.0,0.36,74.467
2283,1.71,64.0,-1.0,21.887076,47.190,32.111013,32.003449,4.0,4.0,0.74,73.753
2284,1.66,57.0,1.0,20.685150,52.940,2.000000,30.854000,4.0,4.0,0.54,76.703
2285,1.69,75.0,-1.0,26.259585,47.010,32.111013,31.300000,3.0,4.0,0.36,75.484


In [6]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 0 to 2286
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   height        2287 non-null   float64
 1   weight        2287 non-null   float64
 2   gender        2287 non-null   float64
 3   bmi           2287 non-null   float64
 4   rh            2287 non-null   float64
 5   stress_level  2287 non-null   float64
 6   skin_temp     2287 non-null   float64
 7   TC            2287 non-null   float64
 8   TS            2287 non-null   float64
 9   Clo           2287 non-null   float64
 10  env_temp      2287 non-null   float64
dtypes: float64(11)
memory usage: 196.7 KB


## data process

### X & y

In [8]:
X_data = df_selected.iloc[:, :-1]

In [9]:
X_cols_name = X_data.columns

In [10]:
y_data = df_selected['env_temp']

### normalize

In [11]:
X_array_normalize, X_norm = normalize(X_data, axis=0, return_norm=True)

In [12]:
X_data = pd.DataFrame(X_array_normalize, columns=X_cols_name)

In [13]:
X_data

,height,weight,gender,bmi,rh,stress_level,skin_temp,TC,TS,Clo
0,0.019628,0.017184,0.020911,0.019573,0.025272,0.010714,0.020906,0.021456,0.018949,0.023861
1,0.021714,0.018984,-0.020911,0.017669,0.019541,0.001099,0.020206,0.021456,0.018949,0.014082
2,0.020978,0.020948,-0.020911,0.020889,0.019256,0.018132,0.021687,0.021456,0.018949,0.023861
3,0.021714,0.018984,-0.020911,0.017669,0.021043,0.029121,0.019878,0.021456,0.018949,0.014082
4,0.021714,0.018984,-0.020911,0.017669,0.022628,0.043956,0.019654,0.021456,0.018949,0.014082
...,...,...,...,...,...,...,...,...,...,...
2282,0.021714,0.029458,-0.020911,0.027417,0.023833,0.018132,0.020481,0.021456,0.018949,0.014082
2283,0.020978,0.020948,-0.020911,0.020889,0.018935,0.017644,0.020906,0.021456,0.018949,0.028947
2284,0.020364,0.018657,0.020911,0.019742,0.021242,0.001099,0.020155,0.021456,0.018949,0.021123
2285,0.020732,0.024549,-0.020911,0.025062,0.018862,0.017644,0.020446,0.016092,0.018949,0.014082


In [14]:
X_norm

array([  81.51502009, 3055.17450402,   47.8225888 , 1047.78619992,
       2492.26763506, 1819.98324996, 1530.85355015,  186.42424735,
        211.09713404,   25.56422109])

## random forest (1st stry)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=1208)

In [16]:
rfr = RandomForestRegressor()

In [17]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [18]:
y_pred = rfr.predict(X_test)

In [19]:
# mean squared error
mean_squared_error(y_test, y_pred)

1.6620547192440513

In [20]:
# R2 score
r2_score(y_test, y_pred)

0.6705262254156985